In [1]:
%pip install --upgrade pip
%pip install huggingface_hub langchain llama-cpp-python

#If you are using Apple Silicon, run the following command in the terminal.
# CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from huggingface_hub import snapshot_download
model_id="BLACKBUN/llama-2-7b-pubmed-qa-211k-gguf"
snapshot_download(repo_id=model_id, local_dir="llama-2-pubmed",
                  local_dir_use_symlinks=False, revision="main")

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 2 files: 100%|██████████| 2/2 [05:56<00:00, 178.28s/it]


'/Users/jinho/Desktop/Medical_LLM/inference/7b/7b_baseline/llama-2-pubmed'

In [6]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain


template = """### Instruction:
Use the Input below to create an instruction, which could have been used to generate the response using an LLM.

### Input:
{question} What is the most likely diagnosis? 

### Response:
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

llm = LlamaCpp(
	# model_path: 로컬머신에 다운로드 받은 모델의 위치
    model_path="./llama-2-pubmed/llama-2-7b-pubmed-qa-211k.gguf",
    temperature=0.5,
    top_p=1,
    max_tokens=8192,
    verbose=True,
    # n_ctx: 모델이 한 번에 처리할 수 있는 최대 컨텍스트 길이
    n_ctx=4096 
)

llm_chain = LLMChain(prompt=prompt, llm=llm)

llama_model_loader: loaded meta data with 15 key-value pairs and 291 tensors from ./llama-2-pubmed/llama-2-7b-pubmed-qa-211k.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q8_0     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q8_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q8_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q8_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q8_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q8_0     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q8_0     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q8_0     [ 11008,  

In [3]:
question = """
A 3-year-old boy is brought to the clinic by his mother who reports that the child has been inconsolable from pain for the last 4 hours. The pain is located in the lower right quadrant of his abdomen. The mother also noticed a bulge in the same area. The child has a history of recurrent constipation. On physical examination, a tender, non-reducible mass is felt in the right inguinal region. What is the most likely diagnosis? """

response = llm_chain.run(prompt.format(question=question))
print(response)


The patient's symptoms are consistent with acute appendicitis. The mother's report of recurrent constipation and the tender, non-reducible mass in the right inguinal region support this diagnosis. A CT scan is ordered to confirm this diagnosis.




llama_print_timings:        load time = 10057.03 ms
llama_print_timings:      sample time =    44.81 ms /    60 runs   (    0.75 ms per token,  1338.90 tokens per second)
llama_print_timings: prompt eval time = 176926.05 ms /   194 tokens (  911.99 ms per token,     1.10 tokens per second)
llama_print_timings:        eval time = 28772.94 ms /    59 runs   (  487.68 ms per token,     2.05 tokens per second)
llama_print_timings:       total time = 205942.32 ms


In [7]:
question = """A 12-year-old boy is brought in by his parents due to recurrent episodes of abdominal pain for the past six months. The pain is described as diffuse and crampy. The episodes usually last for an hour or two and there is no specific timing. The patient denies any nausea, vomiting, diarrhea, or fever. His diet and activity levels have been unchanged. His physical examination is unremarkable, and he appears comfortable between episodes. The pain is not associated with any stressors at school or home. The rest of the review of systems is negative."""
response = llm_chain.run(prompt.format(question=question))
print(response)

The most likely diagnosis is irritable bowel syndrome (IBS). The symptoms are consistent with IBS and there is no other explanation for the abdominal pain. A complete blood count, electrolyte panel, and stool studies may be helpful in the workup of this patient.




llama_print_timings:        load time =  6757.35 ms
llama_print_timings:      sample time =    47.02 ms /    65 runs   (    0.72 ms per token,  1382.30 tokens per second)
llama_print_timings: prompt eval time = 36103.34 ms /   235 tokens (  153.63 ms per token,     6.51 tokens per second)
llama_print_timings:        eval time = 13829.40 ms /    64 runs   (  216.08 ms per token,     4.63 tokens per second)
llama_print_timings:       total time = 50145.35 ms
